In [1]:
from thermo import ChemicalConstantsPackage
from thermo.unifac import UNIFAC, DOUFSG, DOUFIP2016
from chemicals import search_chemical
import numpy as np
import pandas as pd
from IPython.display import clear_output
import sys
sys.path.insert(0,'/home/garren/HPC Files')
from All_code import subsets

In [2]:
subset_df, subset_Indices, subset_Indices_T, Info_Indices, init_indices, init_indices_T = subsets(np.array(['all'])).get_subset_df()

In [3]:
c_all = Info_Indices['Component names']['IUPAC']
c_all_idx = Info_Indices['Component names']['Index']

In [4]:
Idx_known = np.column_stack([subset_df['Component 1 - Index'].to_numpy()[subset_Indices_T[:,0]], 
                             subset_df['Component 2 - Index'].to_numpy()[subset_Indices_T[:,1]]])

del subset_df, subset_Indices, subset_Indices_T, Info_Indices, init_indices, init_indices_T

In [5]:
N = len(c_all)
# Get the excess enthalpy data
testing_excel = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/TestingData_Final.xlsx'
data_df = pd.read_excel(testing_excel)

c1 = data_df['Component 1'].to_numpy().astype(str)
c2 = data_df['Component 2'].to_numpy().astype(str)

idx1 = np.sum(c1[:, np.newaxis] == c_all[np.newaxis,:], axis=1)
idx2 = np.sum(c2[:, np.newaxis] == c_all[np.newaxis,:], axis=1)

idx = (idx1 + idx2) == 2

_, iidx = np.unique(np.char.add(np.char.add(c1[idx], ' + '), c2[idx]), return_index=True)
iidx = np.sort(iidx)

idx1 = np.sum((np.arange(N)[:, np.newaxis] * (c_all[:, np.newaxis]==c1[idx][iidx][np.newaxis,:])), axis=0)
idx2 = np.sum((np.arange(N)[:, np.newaxis] * (c_all[:, np.newaxis]==c2[idx][iidx][np.newaxis,:])), axis=0)

testing_indices = np.column_stack([idx1, idx2])

del N, testing_excel, data_df, c1, c2, idx1, idx2, idx, iidx

In [17]:
testing_indices = testing_indices[np.sum(np.char.add(np.char.add(testing_indices[:,0].astype(str), ' + '), testing_indices[:,1].astype(str))[:,np.newaxis] == np.char.add(np.char.add(Idx_known[:,0].astype(str), ' + '), Idx_known[:,1].astype(str))[np.newaxis,:], axis=1)==0,:]

In [21]:
Idx_all = np.vstack([Idx_known, testing_indices])

In [30]:
Idx_unknown = np.array([[i, j] for i in range(len(c_all)) for j in range(i+1, len(c_all))])
idx_keep = np.sum(np.char.add(np.char.add(Idx_unknown[:,0].astype(str), ' + '), Idx_unknown[:,1].astype(str))[:,np.newaxis] == np.char.add(np.char.add(Idx_all[:,0].astype(str), ' + '), Idx_all[:,1].astype(str))[np.newaxis,:], axis=1) == 0
Idx_unknown = Idx_unknown[idx_keep,:]
del idx_keep

In [35]:
c1_unknown = c_all[Idx_unknown[:,0]]
c2_unknown = c_all[Idx_unknown[:,1]]
x2_int = np.concatenate([np.append(np.linspace(0,0.45, 10)[1:], [0.495, 1-0.495]), np.linspace(0.55, 1, 10)[:-1]])
T2_int = np.array([288.15, 298.15, 308.15])
N_T = len(T2_int)
N_C = len(x2_int)

In [48]:
y = []
for i in range(len(c1_unknown)):
    print(f'Datapoint {i+1} out of {len(c1_unknown)}')
    c1 = c1_unknown[i]
    c2 = c2_unknown[i]
    IUPAC_mix = [c1, c2]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix)
    all_chem_mix = constants.UNIFAC_Dortmund_groups
    for T in T2_int:
        for x in x2_int:
            xs = [x, 1-x]
            GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix, version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
            y += [GE.HE()]
            clear_output(wait=True)

c1_all = [[c1]*N_T*N_C for c1 in c1_unknown]
c2_all = [[c2]*N_T*N_C for c2 in c2_unknown]
T_all = [np.concatenate([T*np.ones_like(x2_int) for T in T2_int]) for _ in range(len(c1_unknown))]
x_all = [np.concatenate([x2_int for _ in range(N_T)]) for _ in range(len(c1_unknown))]

c1_all = np.concatenate(c1_all)
c2_all = np.concatenate(c2_all)
T_all = np.concatenate(T_all)
x_all = np.concatenate(x_all)
y = np.array(y)

Datapoint 3665 out of 3665


In [53]:
df = pd.DataFrame({'Component 1': c1_all,
                   'Component 2': c2_all,
                   'Temperature [K]': T_all,
                   'Composition component 1 [mol/mol]': x_all,
                   'UNIFAC_DMD [J/mol]': y})

In [55]:
df.to_excel('/home/garren/HPC Files/Thermo_UNIFAC_DMD_unknown.xlsx', index=False, header=True)